# 自动求导


In [15]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torchvision import models
x = Variable(torch.Tensor([3]),requires_grad=True)
y = Variable(torch.Tensor([5]),requires_grad=True)
a = x+y
print('a.requires_grad:',a.requires_grad)
b = a+z
print('b.requires_grad:',b.requires_grad)

z = 2*x+y+4
print('z:',z.data)
z.backward()
print('dz/dx:{}'.format(x.grad.data))

a.requires_grad: True
b.requires_grad: True
z: 
 15
[torch.FloatTensor of size 1]

dz/dx:
 2
[torch.FloatTensor of size 1]



In [ ]:
x = 

In [2]:
# 冻结部分网络,只fine tune最后一层
model = torchvision.models.resnet18(pretrained=True)

from param in model.parameyers():
     param.requires_grad = False
model.fc = nn.Linear(512,100)

#Optimize only the classifier
optimizer = optim.SGD(model.fc.parameters(),lr=1e-2,momentum=0.9)


SyntaxError: invalid syntax (<ipython-input-2-56ec94fa7947>, line 4)

In [ ]:

j = Variable(torch.randn(5,5), volatile=True)
k = Variable(torch.randn(5,5))
m = Variable(torch.randn(5,5))
n = k+m # k,m变量的volatile标记都为False，输出的Variable的volatile标记也为false
print(n.volatile)
 
#False
o = j+k #k,m变量的volatile标记有一个True，输出的Variable的volatile为True
print(o.volatile)

# 自定义 Dataset 和输入流

In [ ]:
在输入流水线中，我们看到准备数据的代码是这么写的data = datasets.CIFAR10(“./data/”, transform=transform, train=True, download=True)。
datasets.CIFAR10就是一个Datasets子类，data是这个类的一个实例。
PyTorch提供了一个工具函数torch.utils.data.DataLoader。
通过这个类，我们在准备mini-batch的时候可以多线程并行处理，这样可以加快准备数据的速度。
Datasets就是构建这个类的实例的参数之一。

 ## 如何自定义DataSets框架

In [ ]:
class CustomDataSet(data.Data.Dataset): # 需要继承data.Dataset
    def __init__(self):
        #1.Initialize file path or list of file names
        pass
    def __getitem__(self,index):
        #Read one data from file (e.g. using numpy.fromfile, PIL.Image.open).
        #Process the data (e.g. torchvision.Transform)
        #Return a data pair  (eg. )
    def __len__(self):
        # You should change 0 to the total size of your dataset.
        return 0
         

In [ ]:
class MNIST(data.Dataset):
      def __init__(self, root, train=True, transform=None, target_transform=None, download=False):
        self.root = root
        self.transform = transform
        self.target_transform = target_transform
        self.train = train  # training set or test set
 
        if download:
            self.download()
 
        if not self._check_exists():
            raise RuntimeError('Dataset not found.' +
                               ' You can use download=True to download it')
 
        if self.train:
            self.train_data, self.train_labels = torch.load(
                os.path.join(root, self.processed_folder, self.training_file))
        else:
            self.test_data, self.test_labels = torch.load(os.path.join(root, self.processed_folder, self.test_file))
 
    def __getitem__(self, index):
        if self.train:
            img, target = self.train_data[index], self.train_labels[index]
        else:
            img, target = self.test_data[index], self.test_labels[index]
 
        # doing this so that it is consistent with all other datasets
        # to return a PIL Image
        img = Image.fromarray(img.numpy(), mode='L')
 
        if self.transform is not None:
            img = self.transform(img)
 
        if self.target_transform is not None:
            target = self.target_transform(target)
 
        return img, target
 
    def __len__(self):
        if self.train:
            return 60000
        else:
            return 10000

# 保存和恢复模型并查看参数


In [ ]:
# 方法一 ：只保留和恢复模型中的参数
#保存
torch.save(the_model.state_dict(),PATH)
#恢复
the_model = TheModelClass(*args,**kwargs)
the_model.load_state_dict(torch.load(PATH))

In [ ]:
使用这种方法，将会保存模型的参数和结构信息。

保存
torch.save(the_model, PATH)
恢复
the_model = torch.load(PATH)

# 获取模型某些层的参数

In [8]:
from collections import OrderedDict
model = nn.Sequential(
  OrderedDict([
                  ('conv1', nn.Conv2d(1,20,5)),
                  ('relu1', nn.ReLU()),
                  ('conv2', nn.Conv2d(20,64,5)),
                  ('relu2', nn.ReLU())
                ]))
print(model)

Sequential(
  (conv1): Conv2d (1, 20, kernel_size=(5, 5), stride=(1, 1))
  (relu1): ReLU()
  (conv2): Conv2d (20, 64, kernel_size=(5, 5), stride=(1, 1))
  (relu2): ReLU()
)


In [9]:
params = model.state_dict()
for k,v in params.items():
    print(k)
print(params['conv1.weight'])
print(params['conv1.bias'])

conv1.weight
conv1.bias
conv2.weight
conv2.bias

(0 ,0 ,.,.) = 
 -0.1105  0.0009  0.1988  0.1475 -0.1883
 -0.1840  0.0199 -0.0836 -0.1885  0.0613
 -0.1801 -0.1566  0.0467  0.0208  0.1637
 -0.0894 -0.1378  0.1307  0.0099 -0.0188
 -0.1206 -0.1422  0.1262  0.0902  0.1698

(1 ,0 ,.,.) = 
  0.1467  0.0193  0.1768 -0.1133 -0.0211
  0.0293 -0.1089  0.1337 -0.1058 -0.0195
 -0.0919 -0.0139  0.0615 -0.1580  0.0229
  0.1355 -0.1816 -0.0501  0.0507  0.1734
  0.0392 -0.0055 -0.1916 -0.0774 -0.0697

(2 ,0 ,.,.) = 
  0.1685  0.1635 -0.0603 -0.0741  0.1224
  0.1761  0.1706  0.1626  0.0103 -0.0577
  0.1881  0.1258  0.1454  0.0687 -0.0548
  0.1328  0.1158  0.0935  0.0280 -0.1618
  0.0534  0.0867  0.1006  0.0194  0.1661

(3 ,0 ,.,.) = 
 -0.1584  0.0699  0.1072 -0.1848 -0.0200
  0.0647  0.1024 -0.1003  0.1211 -0.1114
 -0.1187 -0.0267  0.1995 -0.1793 -0.1412
 -0.0534  0.0324 -0.0229  0.0508 -0.1043
  0.0951  0.1986 -0.1132 -0.0144 -0.1044

(4 ,0 ,.,.) = 
  0.1490  0.0049  0.0218 -0.1682  0.0030
  0.1390  0

# 学习率衰减

In [ ]:
很多时候我们要对学习率（learning rate）进行衰减，下面的代码示范了如何每30个epoch按10%的速率衰减：

def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = args.lr * (0.1 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = args.lr * (0.1 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
什么是param_groups?
optimizer通过param_group来管理参数组.param_group中保存了参数组及其对应的学习率,动量等等.所以我们可以通过更改param_group[‘lr’]的值来更改对应参数组的学习率。

# 有两个`param_group`即,len(optim.param_groups)==2
optim.SGD([
                {'params': model.base.parameters()},
                {'params': model.classifier.parameters(), 'lr': 1e-3}
            ], lr=1e-2, momentum=0.9)

#一个参数组
optim.SGD(model.parameters(), lr=1e-2, momentum=.9)

# 有两个`param_group`即,len(optim.param_groups)==2
optim.SGD([
                {'params': model.base.parameters()},
                {'params': model.classifier.parameters(), 'lr': 1e-3}
            ], lr=1e-2, momentum=0.9)
 
#一个参数组
optim.SGD(model.parameters(), lr=1e-2, momentum=.9)